In [1]:
import re
import json

In [2]:
def synonyms(text):
    for c, line in enumerate(text.splitlines()):
        if line=='==== {{S|synonymes}} ====':
            text_=text.splitlines()[c+1:]
            synonyms = []
            for line in text_:
                synonyms.append(line)
                if line.startswith('='):
                    break
            syn = []
            for syno in synonyms:
                ree = re.findall('\[[^\[\]]*\]', syno)
                for r in ree:
                    r = r.replace('[','').replace(']','')
                    if '|' in r:
                        r=r.split('|')[1]
                    syn.append(r)
            return syn

In [3]:
import xml.sax

class WikiHandler( xml.sax.ContentHandler ):
    
    def __init__(self):
        self.page = False

        self.text = None
        self.title = None
        
        self.synonyms = {}
        self.all_content = None
        
    
    def startElement(self, tag, attributes):
        if tag == "page":
            self.page = True
            
        if self.page and tag in ("title", "text"):
            self.all_content = []
        else:
            self.all_content = None

    def endElement(self, tag):
        if tag == "page":
            self.page = False
            self.all_content = None

        if self.all_content is not None:
            content = ''.join(self.all_content)
            if tag == 'title':
                self.title = content
            elif tag == 'text':
                self.text = content
            self.all_content = None

                
        if self.title and self.text:
            if self.text.startswith('== {{langue|fr}} =='):
                self.synonyms[self.title] = synonyms(self.text)
                if self.synonyms[self.title]:
                    print(json.dumps({self.title: self.synonyms[self.title]}), file=self.file)
        
            self.title = None
            self.text = None

        
    def characters(self, content):
        if self.all_content is not None:
            self.all_content.append(content)   

In [4]:
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
Handler = WikiHandler()
with open('synonyms.txt', 'w') as f:
    Handler.file = f
    parser.setContentHandler( Handler )


    parser.parse("/Users/liudmylaslava/Downloads/frwiktionary-20200301-pages-meta-current.xml")

In [5]:
!wc -l synonyms.txt

   36463 synonyms.txt
